In [2]:
import numpy as np
from core import Psi4Model, XYZLogger #, load_xyz
import psi4

#https://github.com/molmod/molmod
from molmod.units import angstrom, debye
from molmod.io.xyz import XYZReader

import matplotlib.pyplot as plt
import os, sys

In [3]:
numbers = np.array([1, 1])
# De coordinaten die uit de paper zelf komen, ze staan in angstrom
positions = np.array([[0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00],
                      [0.734900000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00]])

In [6]:
numbers = np.array([3, 1])
# De coordinaten die uit de paper zelf komen, ze staan in angstrom
positions = np.array([[0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00],
                      [1.548000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00]])

In [7]:
# Method kan je bv veranderen naar 'pbe0' voor dft berekeningen of 'ccsd' voor coupled cluster, etc...
# cores is het aantal cpu cores dat je wil gebruiken
# memory je computer geheugen dat je beschikbaar stelt aan psi4
model = Psi4Model(method = 'scf', basis = 'STO-3G', psi4_output = 'output.dat', cores=2, memory=2e+09)

# Eenmaal het classobject is aangemaakt, kan je nu verschillende keren de compute functie callen.
energy = model.compute(numbers, positions, field_type = 'dipole', dipole_strength = [0, 0, 0], charge = 0, forces = False, uhf=False)

Psi4 Model initiated with method scf and basis set STO-3G


In [23]:
energy - model.mol.nuclear_repulsion_energy()

-53.01801776079779

In [8]:
energy

-7.863190101746813

In [51]:
model.mol.nuclear_repulsion_energy()

0.720066962403048

In [16]:
mints = psi4.core.MintsHelper(model.wavefunction)
C_occ = np.array(model.wavefunction.Ca())

In [31]:
eri = np.array(mints.mo_eri(model.wavefunction.Ca(),model.wavefunction.Ca(),model.wavefunction.Ca(),model.wavefunction.Ca()))
sp_eri = 0.5*np.array(mints.mo_spin_eri(model.wavefunction.Ca(),model.wavefunction.Cb()))

In [89]:
tb_int = 0.5*np.einsum('ijkl->iklj',np.array(mints.ao_eri()))
ob_int = np.array(mints.ao_kinetic()) + np.array(mints.ao_potential())

In [88]:
mints = psi4.core.MintsHelper(model.wavefunction.basisset())
C_occ = np.array(model.wavefunction.Ca())

In [90]:
tb_int = np.einsum('ia,jb,ijkl,kc,ld->abcd', C_occ, C_occ, tb_int, C_occ, C_occ, optimize=True)
ob_int = np.einsum('ia,il,ld->ad', C_occ, ob_int, C_occ, optimize=True)
ob_int = np.where(np.abs(ob_int)>1e-15, ob_int, 0)

In [92]:
OB = np.load('../data/integrals/STO-3G_LiH_ob.npy')
TB = np.load('../data/integrals/STO-3G_LiH_tb.npy')

In [24]:
OB = np.load('../data/integrals/STO-3G_H2_ob.npy')
TB = np.load('../data/integrals/STO-3G_H2_tb.npy')

In [188]:
energy - model.mol.nuclear_repulsion_energy()

-1.8369920444380115

## Equilibrium geomtries (STO-3G calculated)

In [27]:
numbers = np.array([1, 1])
# De coordinaten die uit de paper zelf komen, ze staan in angstrom
positions = np.array([[0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00],
                      [0.734000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00]])

In [125]:
numbers = np.array([3, 1])
# De coordinaten die uit de paper zelf komen, ze staan in angstrom
positions = np.array([[0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00],
                      [1.548000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00]])

In [63]:
numbers = np.array([6, 6])
# De coordinaten die uit de paper zelf komen, ze staan in angstrom
positions = np.array([[0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00],
                      [1.260000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00]])

In [65]:
numbers = np.array([7, 7])
# De coordinaten die uit de paper zelf komen, ze staan in angstrom
positions = np.array([[0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00],
                      [1.190000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00]])

In [18]:
numbers = np.array([6, 1, 1, 1, 1])
# De coordinaten die uit de paper zelf komen, ze staan in angstrom
positions = np.array([[0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00],
                      [0.640000000000000e+00, 0.640000000000000e+00, 0.640000000000000e+00],
                      [-0.640000000000000e+00, -0.640000000000000e+00, 0.640000000000000e+00],
                      [-0.640000000000000e+00, 0.640000000000000e+00, -0.640000000000000e+00],
                      [0.640000000000000e+00, -0.640000000000000e+00, -0.640000000000000e+00]])

In [66]:
generate_integrals(numbers, positions, eq=1)

Psi4 Model initiated with method FCI and basis set STO-3G
writing OB integral: ../data/integrals/STO-3G/STO-3G_N2_OB_d1-19_eq1.npy
writing TB integral: ../data/integrals/STO-3G/STO-3G_N2_TB_d1-19_eq1.npy


(-107.67737119367379, 21.789649851117648)

## Generate datadictionary

In [3]:
energy_data = np.load('../data/energies/STO-3G_H2_energy.npy')

In [5]:
np.shape(energy_data)

(3, 9)

In [2]:
sto3g_H2 = []
energy_data = np.load('../data/energies/STO-3G_H2_energy.npy').transpose()
for dat in energy_data:
    mol_dict = {'molecule' : 'H2',
                'basis' : 'sto3g',
                'n_basisfuncs' : 4,
                'distance' : dat[0],
                'atomstring' : 'H 0.0 0.0 0.0; H 0.0 0.0 %f' % dat[0],
                'eq' : False,
                'tot_energy' : dat[1],
                'nuc_rep_energy' : dat[2],
                'n_electrons' : 2}
    sto3g_H2.append(mol_dict)
sto3g_H2[3]['eq'] = True

sto3g_H2_eq = sto3g_H2[3]

b631g_H2 = []
energy_data = np.load('../data/energies/6-31g_H2_energy.npy').transpose()
for dat in energy_data:
    mol_dict = {'molecule' : 'H2',
                'basis' : '631g',
                'n_basisfuncs' : 8,
                'distance' : dat[0],
                'atomstring' : 'H 0.0 0.0 0.0; H 0.0 0.0 %f' % dat[0],
                'eq' : False,
                'tot_energy' : dat[1],
                'nuc_rep_energy' : dat[2],
                'n_electrons' : 2}
    b631g_H2.append(mol_dict)
b631g_H2[3]['eq'] = True

b631g_H2_eq = b631g_H2[3]

sto3g_LiH = []
energy_data = np.load('../data/energies/STO-3G_LiH_energy.npy').transpose()
for dat in energy_data:
    mol_dict = {'molecule' : 'LiH',
                'basis' : 'sto3g',
                'n_basisfuncs' : 12,
                'distance' : dat[0],
                'atomstring' : 'Li 0.0 0.0 0.0; H 0.0 0.0 %f' % dat[0],
                'eq' : False,
                'tot_energy' : dat[1],
                'nuc_rep_energy' : dat[2],
                'n_electrons' : 4}
    sto3g_LiH.append(mol_dict)
sto3g_LiH[3]['eq'] = True

sto3g_LiH_eq = sto3g_LiH[3]

b631g_LiH = []
energy_data = np.load('../data/energies/6-31g_LiH_energy.npy').transpose()
for dat in energy_data:
    mol_dict = {'molecule' : 'LiH',
                'basis' : '631g',
                'n_basisfuncs' : 22,
                'distance' : dat[0],
                'atomstring' : 'Li 0.0 0.0 0.0; H 0.0 0.0 %f' % dat[0],
                'eq' : False,
                'tot_energy' : dat[1],
                'nuc_rep_energy' : dat[2],
                'n_electrons' : 4}
    b631g_LiH.append(mol_dict)
b631g_LiH[4]['eq'] = True

b631g_LiH_eq = b631g_LiH[4]

sto3g_C2 = []
energy_data = np.load('../data/energies/STO-3G_C2_energy.npy').transpose()
for dat in energy_data:
    mol_dict = {'molecule' : 'C2',
                'basis' : 'sto3g',
                'n_basisfuncs' : 20,
                'distance' : dat[0],
                'atomstring' : 'C 0.0 0.0 0.0; C 0.0 0.0 %f' % dat[0],
                'eq' : False,
                'tot_energy' : dat[1],
                'nuc_rep_energy' : dat[2],
                'n_electrons' : 12}
    sto3g_C2.append(mol_dict)
sto3g_C2[5]['eq'] = True

sto3g_C2_eq = sto3g_C2[5]

b631g_C2_eq = {'molecule' : 'C2',
            'basis' : '631g',
            'n_basisfuncs' : 36,
            'distance' : 1.261,
            'atomstring' : 'C 0.0 0.0 0.0; C 0.0 0.0 1.261',
            'eq' : True,
            'tot_energy' : -75.623194,
            'nuc_rep_energy' : 15.10735891,
            'n_electrons' : 12}

sto3g_N2 = []
energy_data = np.load('../data/energies/STO-3G_N2_energy.npy').transpose()
for dat in energy_data:
    mol_dict = {'molecule' : 'N2',
                'basis' : 'sto3g',
                'n_basisfuncs' : 20,
                'distance' : dat[0],
                'atomstring' : 'N 0.0 0.0 0.0; N 0.0 0.0 %f' % dat[0],
                'eq' : False,
                'tot_energy' : dat[1],
                'nuc_rep_energy' : dat[2],
                'n_electrons' : 14}
    sto3g_N2.append(mol_dict)
sto3g_N2[3]['eq'] = True

sto3g_N2_eq = sto3g_N2[3]

b631g_N2_eq = {'molecule' : 'N2',
            'basis' : '631g',
            'n_basisfuncs' : 36,
            'distance' : 1.333,
            'atomstring' : 'N 0.0 0.0 0.0; N 0.0 0.0 1.1333',
            'eq' : True,
            'tot_energy' : -109.106354,
            'nuc_rep_energy' : 22.87980528,
            'n_electrons' : 14}

sto3g_CH4_eq = {'molecule' : 'CH4',
                'basis' : 'sto3g',
                'n_basisfuncs' : 18,
                'distance' : 0,
                'atomstring' : 'C 0.0 0.0 0.0; H 0.640 0.640 0.640; H -0.640 -0.640 0.640; H -0.640 0.640 -0.640; H 0.640 -0.640 -0.640',
                'eq' : True,
                'tot_energy' : -39.807003849786966,
                'nuc_rep_energy' : 13.211013911010273,
                'n_electrons' : 10}

## Dissociation curve function

In [80]:
def generate_dissociation_data(steps_neg, steps_pos, stepsize, r_eq, numbers, basisset = 'STO-3G'):
    molname = ''
    elem_numbers = np.array([16,8,7,6,5,3,1])
    elem_names = ['S','O','N','C','B','Li','H']
    for i,el in enumerate(elem_numbers):
            n_el = len(np.where(numbers == el)[0])
            if n_el > 1:
                molname += elem_names[i] + str(n_el)
            elif n_el != 0:
                molname += elem_names[i]
    
    bondlength = np.arange(-steps_neg*stepsize, steps_pos*stepsize, stepsize) + r_eq
    print('saving: ../data/dissociation/' + basisset + '/bondlength_' + molname + '.npy')
    np.save('../data/dissociation/' + basisset + '/bondlength_' + molname + '.npy', bondlength)
    
    energy = np.zeros((len(bondlength)))
    positions = np.zeros((2,3))
    lot = ['scf', 'FCI', 'CCSD', 'CCSD(T)']
    for l in lot:
        model = Psi4Model(method = l, basis = basisset, cores=2, memory=2e+09)
        for i, bl in enumerate(bondlength):
            positions[1,0] = bl
            energy[i] = model.compute(numbers, positions, charge = 0)
        print('saving: ../data/dissociation/' + basisset + '/' + l + '_energy_' + molname + '.npy')
        np.save('../data/dissociation/' + basisset + '/' + l + '_energy_' + molname + '.npy', energy)

#### H2

In [130]:
generate_dissociation_data(steps_neg=10, steps_pos=20, stepsize=0.03, r_eq=0.7349, numbers=np.array([1,1]))

saving: ../data/dissociation/STO-3G/bondlength_H2.npy
Psi4 Model initiated with method scf and basis set STO-3G
saving: ../data/dissociation/STO-3G/scf_energy_H2.npy
Psi4 Model initiated with method FCI and basis set STO-3G
saving: ../data/dissociation/STO-3G/FCI_energy_H2.npy
Psi4 Model initiated with method CCSD and basis set STO-3G
saving: ../data/dissociation/STO-3G/CCSD_energy_H2.npy
Psi4 Model initiated with method CCSD(T) and basis set STO-3G
saving: ../data/dissociation/STO-3G/CCSD(T)_energy_H2.npy


In [105]:
generate_dissociation_data(steps_neg=11, steps_pos=19, stepsize=0.03, r_eq=0.745882, numbers=np.array([1,1]), basisset='6-31g')

saving: ../data/dissociation/6-31g/bondlength_H2.npy
Psi4 Model initiated with method scf and basis set 6-31g
saving: ../data/dissociation/6-31g/scf_energy_H2.npy
Psi4 Model initiated with method FCI and basis set 6-31g
saving: ../data/dissociation/6-31g/FCI_energy_H2.npy
Psi4 Model initiated with method CCSD and basis set 6-31g
saving: ../data/dissociation/6-31g/CCSD_energy_H2.npy
Psi4 Model initiated with method CCSD(T) and basis set 6-31g
saving: ../data/dissociation/6-31g/CCSD(T)_energy_H2.npy


#### LiH

In [141]:
generate_dissociation_data(steps_neg=10, steps_pos=20, stepsize=0.05, r_eq=1.5475, numbers=np.array([3,1]))

saving: ../data/dissociation/STO-3G/bondlength_LiH.npy
Psi4 Model initiated with method scf and basis set STO-3G
saving: ../data/dissociation/STO-3G/scf_energy_LiH.npy
Psi4 Model initiated with method FCI and basis set STO-3G
saving: ../data/dissociation/STO-3G/FCI_energy_LiH.npy
Psi4 Model initiated with method CCSD and basis set STO-3G
saving: ../data/dissociation/STO-3G/CCSD_energy_LiH.npy
Psi4 Model initiated with method CCSD(T) and basis set STO-3G
saving: ../data/dissociation/STO-3G/CCSD(T)_energy_LiH.npy


In [81]:
generate_dissociation_data(steps_neg=12, steps_pos=18, stepsize=0.05, r_eq=1.6721, numbers=np.array([3,1]), basisset='6-31g')

saving: ../data/dissociation/6-31g/bondlength_LiH.npy
Psi4 Model initiated with method scf and basis set 6-31g
saving: ../data/dissociation/6-31g/scf_energy_LiH.npy
Psi4 Model initiated with method FCI and basis set 6-31g
saving: ../data/dissociation/6-31g/FCI_energy_LiH.npy
Psi4 Model initiated with method CCSD and basis set 6-31g
saving: ../data/dissociation/6-31g/CCSD_energy_LiH.npy
Psi4 Model initiated with method CCSD(T) and basis set 6-31g
saving: ../data/dissociation/6-31g/CCSD(T)_energy_LiH.npy


#### C2

In [56]:
generate_dissociation_data(steps_neg=15, steps_pos=8, stepsize=0.03, r_eq=1.26, numbers=np.array([6,6]))

saving: ../data/dissociation/STO-3G/bondlength_C2.npy
Psi4 Model initiated with method scf and basis set STO-3G
saving: ../data/dissociation/STO-3G/scf_energy_C2.npy
Psi4 Model initiated with method FCI and basis set STO-3G
saving: ../data/dissociation/STO-3G/FCI_energy_C2.npy
Psi4 Model initiated with method CCSD and basis set STO-3G
saving: ../data/dissociation/STO-3G/CCSD_energy_C2.npy
Psi4 Model initiated with method CCSD(T) and basis set STO-3G
saving: ../data/dissociation/STO-3G/CCSD(T)_energy_C2.npy


In [ ]:
generate_dissociation_data(steps_neg=15, steps_pos=8, stepsize=0.03, r_eq=1.261, numbers=np.array([6,6]), basisset='6-31g')

#### N2

In [155]:
generate_dissociation_data(steps_neg=8, steps_pos=17, stepsize=0.05, r_eq=1.1939, numbers=np.array([7,7]))

saving: ../data/dissociation/STO-3G/bondlength_N2.npy
Psi4 Model initiated with method scf and basis set STO-3G
saving: ../data/dissociation/STO-3G/scf_energy_N2.npy
Psi4 Model initiated with method FCI and basis set STO-3G
saving: ../data/dissociation/STO-3G/FCI_energy_N2.npy
Psi4 Model initiated with method CCSD and basis set STO-3G
saving: ../data/dissociation/STO-3G/CCSD_energy_N2.npy
Psi4 Model initiated with method CCSD(T) and basis set STO-3G
saving: ../data/dissociation/STO-3G/CCSD(T)_energy_N2.npy


## Generating one- and two-body integrals

In [19]:
bondlength = np.load('../data/dissociation/6-31g/bondlength_LiH.npy')

In [20]:
bondlength

array([1.071, 1.121, 1.171, 1.221, 1.271, 1.321, 1.371, 1.421, 1.471,
       1.521, 1.571, 1.621, 1.671, 1.721, 1.771, 1.821, 1.871, 1.921,
       1.971, 2.021, 2.071, 2.121, 2.171, 2.221, 2.271, 2.321, 2.371,
       2.421, 2.471, 2.521])

In [21]:
bondlength[[1,4,7,11,12,13,17,20,23]]

array([1.121, 1.271, 1.421, 1.621, 1.671, 1.721, 1.921, 2.071, 2.221])

In [226]:
def generate_integrals(numbers, positions, basisset = 'STO-3G', dist = True, eq = 0):
    # Method kan je bv veranderen naar 'pbe0' voor dft berekeningen of 'ccsd' voor coupled cluster, etc...
    # cores is het aantal cpu cores dat je wil gebruiken
    # memory je computer geheugen dat je beschikbaar stelt aan psi4
    model = Psi4Model(method = 'scf', basis = basisset, psi4_output = 'output.dat', cores=2, memory=2e+09)

    # Eenmaal het classobject is aangemaakt, kan je nu verschillende keren de compute functie callen.
    energy = model.compute(numbers, positions, charge = 0)
    
    mints = psi4.core.MintsHelper(model.wavefunction.basisset())
    
    tb_int = np.array(mints.ao_eri())
    ob_int = np.array(mints.ao_kinetic()) + np.array(mints.ao_potential())
    
    molname = ''
    elem_numbers = np.array([16,8,7,6,5,3,1])
    elem_names = ['S','O','N','C','B','Li','H']
    for i,el in enumerate(elem_numbers):
            n_el = len(np.where(numbers == el)[0])
            if n_el > 1:
                molname += elem_names[i] + str(n_el)
            elif n_el != 0:
                molname += elem_names[i]
    
    ob_filename = '../data/integrals/' + basisset + '/' + basisset + '_' + molname + '_OB_'
    tb_filename = '../data/integrals/' + basisset + '/' + basisset + '_' + molname + '_TB_'
    
    if dist:
        dist = positions[1,0]
        ob_filename += 'd' + str(dist).replace('.','-')[:6] + '_'
        tb_filename += 'd' + str(dist).replace('.','-')[:6] + '_'
    
    ob_filename += 'eq' + str(eq) + '.npy'
    tb_filename += 'eq' + str(eq) + '.npy'
    
    print('writing OB integral: ' + ob_filename)
    np.save(ob_filename, ob_int)
    print('writing TB integral: ' + tb_filename)
    np.save(tb_filename, tb_int)
    
    return energy, model.mol.nuclear_repulsion_energy()

In [19]:
numbers = np.array([6, 6])
# De coordinaten die uit de paper zelf komen, ze staan in angstrom
positions = np.array([[0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00],
                      [1.261000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00]])


model = Psi4Model(method = 'scf', basis = 'STO-3G', psi4_output = 'output.dat', cores=2, memory=2e+09)
# Eenmaal het classobject is aangemaakt, kan je nu verschillende keren de compute functie callen.
HF_energy = model.compute(numbers, positions, charge = 0)
E_nr = model.mol.nuclear_repulsion_energy()
energies = np.array([1.261, -75.623194, E_nr])
np.save('../data/energies/6-31g_C2_energy.npy', energies)

Psi4 Model initiated with method scf and basis set STO-3G


In [20]:
numbers = np.array([7, 7])
# De coordinaten die uit de paper zelf komen, ze staan in angstrom
positions = np.array([[0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00],
                      [1.133300000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00]])

model = Psi4Model(method = 'scf', basis = 'STO-3G', psi4_output = 'output.dat', cores=2, memory=2e+09)
# Eenmaal het classobject is aangemaakt, kan je nu verschillende keren de compute functie callen.
HF_energy = model.compute(numbers, positions, charge = 0)
E_nr = model.mol.nuclear_repulsion_energy()
energies = np.array([1.133, -109.106354, E_nr])
np.save('../data/energies/6-31g_N2_energy.npy', energies)

Psi4 Model initiated with method scf and basis set STO-3G


In [14]:
numbers = np.array([1, 1])
# De coordinaten die uit de paper zelf komen, ze staan in angstrom
positions = np.array([[0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00],
                      [0.734000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00]])

# selecting a subset of the bondlengths
bondlength = np.load('../data/dissociation/STO-3G/bondlength_H2.npy')
bondlength = bondlength[[1,5,9,10,11,14,17,22,25]]
energies = np.zeros((3,len(bondlength)))
for i,bl in enumerate(bondlength):
    positions[1,0] = bl
    model = Psi4Model(method = 'FCI', basis = 'STO-3G', psi4_output = 'output.dat', cores=2, memory=2e+09)
    # Eenmaal het classobject is aangemaakt, kan je nu verschillende keren de compute functie callen.
    E_tot = model.compute(numbers, positions, charge = 0)
    energies[0,i] = bl
    energies[1,i] = E_tot
    energies[2,i] = model.mol.nuclear_repulsion_energy()
np.save('../data/energies/STO-3G_H2_energy.npy', energies)

Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G


In [13]:
numbers = np.array([1, 1])
# De coordinaten die uit de paper zelf komen, ze staan in angstrom
positions = np.array([[0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00],
                      [0.734000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00]])

# selecting a subset of the bondlengths
bondlength = np.load('../data/dissociation/6-31g/bondlength_H2.npy')
bondlength = bondlength[[1,4,7,11,12,13,17,20,23]]
energies = np.zeros((3,len(bondlength)))
for i,bl in enumerate(bondlength):
    positions[1,0] = bl
    model = Psi4Model(method = 'FCI', basis = '6-31g', psi4_output = 'output.dat', cores=2, memory=2e+09)
    # Eenmaal het classobject is aangemaakt, kan je nu verschillende keren de compute functie callen.
    E_tot = model.compute(numbers, positions, charge = 0)
    energies[0,i] = bl
    energies[1,i] = E_tot
    energies[2,i] = model.mol.nuclear_repulsion_energy()
np.save('../data/energies/6-31g_H2_energy.npy', energies)

Psi4 Model initiated with method FCI and basis set 6-31g
Psi4 Model initiated with method FCI and basis set 6-31g
Psi4 Model initiated with method FCI and basis set 6-31g
Psi4 Model initiated with method FCI and basis set 6-31g
Psi4 Model initiated with method FCI and basis set 6-31g
Psi4 Model initiated with method FCI and basis set 6-31g
Psi4 Model initiated with method FCI and basis set 6-31g
Psi4 Model initiated with method FCI and basis set 6-31g
Psi4 Model initiated with method FCI and basis set 6-31g


In [12]:
numbers = np.array([3, 1])
# De coordinaten die uit de paper zelf komen, ze staan in angstrom
positions = np.array([[0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00],
                      [1.548000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00]])

# selecting a subset of the bondlengths
bondlength = np.load('../data/dissociation/STO-3G/bondlength_LiH.npy')
bondlength = bondlength[[1,5,9,10,11,14,17,22,25]]
energies = np.zeros((3,len(bondlength)))
for i,bl in enumerate(bondlength):
    positions[1,0] = bl
    model = Psi4Model(method = 'FCI', basis = 'STO-3G', psi4_output = 'output.dat', cores=2, memory=2e+09)
    # Eenmaal het classobject is aangemaakt, kan je nu verschillende keren de compute functie callen.
    E_tot = model.compute(numbers, positions, charge = 0)
    energies[0,i] = bl
    energies[1,i] = E_tot
    energies[2,i] = model.mol.nuclear_repulsion_energy()
np.save('../data/energies/STO-3G_LiH_energy.npy', energies)

Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G


In [82]:
numbers = np.array([3, 1])
# De coordinaten die uit de paper zelf komen, ze staan in angstrom
positions = np.array([[0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00],
                      [1.548000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00]])

# selecting a subset of the bondlengths
bondlength = np.load('../data/dissociation/6-31g/bondlength_LiH.npy')
bondlength = bondlength[[1,4,7,11,12,13,17,20,23]]
energies = np.zeros((3,len(bondlength)))
for i,bl in enumerate(bondlength):
    positions[1,0] = bl
    model = Psi4Model(method = 'FCI', basis = '6-31g', psi4_output = 'output.dat', cores=2, memory=2e+09)
    # Eenmaal het classobject is aangemaakt, kan je nu verschillende keren de compute functie callen.
    E_tot = model.compute(numbers, positions, charge = 0)
    energies[0,i] = bl
    energies[1,i] = E_tot
    energies[2,i] = model.mol.nuclear_repulsion_energy()
np.save('../data/energies/6-31g_LiH_energy.npy', energies)

Psi4 Model initiated with method FCI and basis set 6-31g
Psi4 Model initiated with method FCI and basis set 6-31g
Psi4 Model initiated with method FCI and basis set 6-31g
Psi4 Model initiated with method FCI and basis set 6-31g
Psi4 Model initiated with method FCI and basis set 6-31g
Psi4 Model initiated with method FCI and basis set 6-31g
Psi4 Model initiated with method FCI and basis set 6-31g
Psi4 Model initiated with method FCI and basis set 6-31g
Psi4 Model initiated with method FCI and basis set 6-31g


In [16]:
numbers = np.array([6, 6])
# De coordinaten die uit de paper zelf komen, ze staan in angstrom
positions = np.array([[0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00],
                      [1.260000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00]])

# selecting a subset of the bondlengths
bondlength = np.load('../data/dissociation/STO-3G/bondlength_C2.npy')
bondlength = bondlength[[5,7,9,11,14,15,16,22]]
energies = np.zeros((3,len(bondlength)))
for i,bl in enumerate(bondlength):
    positions[1,0] = bl
    model = Psi4Model(method = 'FCI', basis = 'STO-3G', psi4_output = 'output.dat', cores=2, memory=2e+09)
    # Eenmaal het classobject is aangemaakt, kan je nu verschillende keren de compute functie callen.
    E_tot = model.compute(numbers, positions, charge = 0)
    energies[0,i] = bl
    energies[1,i] = E_tot
    energies[2,i] = model.mol.nuclear_repulsion_energy()
np.save('../data/energies/STO-3G_C2_energy.npy', energies)

Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G


In [17]:
numbers = np.array([7, 7])
# De coordinaten die uit de paper zelf komen, ze staan in angstrom
positions = np.array([[0.000000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00],
                      [1.190000000000000e+00, 0.000000000000000e+00, 0.000000000000000e+00]])

# selecting a subset of the bondlengths
bondlength = np.load('../data/dissociation/STO-3G/bondlength_N2.npy')
bondlength = bondlength[[1,4,7,8,9,12,15,17,20]]
energies = np.zeros((3,len(bondlength)))
for i,bl in enumerate(bondlength):
    positions[1,0] = bl
    model = Psi4Model(method = 'FCI', basis = 'STO-3G', psi4_output = 'output.dat', cores=2, memory=2e+09)
    # Eenmaal het classobject is aangemaakt, kan je nu verschillende keren de compute functie callen.
    E_tot = model.compute(numbers, positions, charge = 0)
    energies[0,i] = bl
    energies[1,i] = E_tot
    energies[2,i] = model.mol.nuclear_repulsion_energy()
np.save('../data/energies/STO-3G_N2_energy.npy', energies)

Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
Psi4 Model initiated with method FCI and basis set STO-3G
